In [10]:
import requests
import pandas as pd

def fetch_binance_ohlcv(symbol="BTCUSDT", interval="5m", limit=50):
    """
    Fetch OHLCV (Open, High, Low, Close, Volume) data from Binance
    """
    url = "https://api.binance.com/api/v3/klines"
    params = {"symbol": symbol, "interval": interval, "limit": limit}
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()
    df = pd.DataFrame(data, columns=[
        "open_time","open","high","low","close","volume",
        "close_time","qav","num_trades","tbbav","tbqav","ignore"
    ])
    df["close"] = df["close"].astype(float)
    return df[["open_time","close"]]

In [11]:
fetch_binance_ohlcv(symbol='BTCUSDT', interval='5m', limit=50)

,open_time,close
0,1757319600000,111493.42
1,1757319900000,111594.94
2,1757320200000,111735.95
3,1757320500000,111729.95
4,1757320800000,111807.35
5,1757321100000,111754.87
6,1757321400000,111623.30
7,1757321700000,111684.25
8,1757322000000,111655.34
9,1757322300000,111843.56


In [14]:
import requests
import pandas as pd

# 🔹 Get BTC/ETH 5m candles from Binance
def fetch_binance_data(symbol="BTCUSDT", interval="5m", limit=10):
    url = "https://api.binance.com/api/v3/klines"
    params = {"symbol": symbol, "interval": interval, "limit": limit}
    resp = requests.get(url, params=params)
    data = resp.json()

    df = pd.DataFrame(data, columns=[
        "open_time", "open", "high", "low", "close", "volume",
        "close_time", "qav", "num_trades",
        "taker_base_vol", "taker_quote_vol", "ignore"
    ])

    # keep only required columns
    df = df[["open_time", "close"]].astype({"close": float})

    # convert open_time → IST
    df["open_time"] = pd.to_datetime(df["open_time"], unit="ms") \
                        .dt.tz_localize("UTC") \
                        .dt.tz_convert("Asia/Kolkata")

    # sort by latest time first
    df = df.sort_values(by="open_time", ascending=False).reset_index(drop=True)
    return df

# ✅ Example usage
btc_df = fetch_binance_data("BTCUSDT", limit=10)
eth_df = fetch_binance_data("ETHUSDT", limit=10)

print("BTC Data (Latest First):")
print(btc_df.head())

BTC Data (Latest First):
                  open_time      close
0 2025-09-08 18:05:00+05:30  112053.50
1 2025-09-08 18:00:00+05:30  112019.88
2 2025-09-08 17:55:00+05:30  111947.40
3 2025-09-08 17:50:00+05:30  111937.52
4 2025-09-08 17:45:00+05:30  111934.24


In [15]:
print("\nETH Data (INR):")
print(eth_df.head())


ETH Data (INR):
                  open_time    close
0 2025-09-08 18:05:00+05:30  4319.00
1 2025-09-08 18:00:00+05:30  4317.37
2 2025-09-08 17:55:00+05:30  4315.70
3 2025-09-08 17:50:00+05:30  4313.03
4 2025-09-08 17:45:00+05:30  4312.09


In [19]:
import requests
import pandas as pd

def fetch_binance_data(symbol="BTCUSDT", interval="5m", limit=100):
    url = "https://api.binance.com/api/v3/klines"
    params = {"symbol": symbol, "interval": interval, "limit": limit}
    resp = requests.get(url, params=params)
    data = resp.json()

    df = pd.DataFrame(data, columns=[
        "open_time", "open", "high", "low", "close", "volume",
        "close_time", "qav", "num_trades",
        "taker_base_vol", "taker_quote_vol", "ignore"
    ])

    df = df[["open_time", "close"]].astype({"close": float})

    df["open_time"] = pd.to_datetime(df["open_time"], unit="ms") \
                        .dt.tz_localize("UTC") \
                        .dt.tz_convert("Asia/Kolkata")

    # oldest → latest (RSI calc sahi hoga)
    df = df.sort_values(by="open_time", ascending=True).reset_index(drop=True)

    df["RSI"] = calculate_rsi(df["close"])

    # recent → oldest for viewing
    df = df.sort_values(by="open_time", ascending=False).reset_index(drop=True)
    return df

def calculate_rsi(series: pd.Series, period: int = 14) -> pd.Series:
    delta = series.diff()

    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_gain = gain.ewm(alpha=1/period, min_periods=period, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/period, min_periods=period, adjust=False).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

# ✅ Example usage
btc_df = fetch_binance_data("BTCUSDT", limit=100)
eth_df = fetch_binance_data("ETHUSDT", limit=100)

print("BTC Latest 5 Rows with RSI:")
print(btc_df.head())

print("\nETH Latest 5 Rows with RSI:")
print(eth_df.head())

BTC Latest 5 Rows with RSI:
                  open_time      close        RSI
0 2025-09-08 18:10:00+05:30  112068.30  57.530685
1 2025-09-08 18:05:00+05:30  112082.33  58.663739
2 2025-09-08 18:00:00+05:30  112019.88  55.000651
3 2025-09-08 17:55:00+05:30  111947.40  50.249307
4 2025-09-08 17:50:00+05:30  111937.52  49.575389

ETH Latest 5 Rows with RSI:
                  open_time    close        RSI
0 2025-09-08 18:10:00+05:30  4320.25  55.642319
1 2025-09-08 18:05:00+05:30  4319.07  54.524304
2 2025-09-08 18:00:00+05:30  4317.37  52.937451
3 2025-09-08 17:55:00+05:30  4315.70  51.390188
4 2025-09-08 17:50:00+05:30  4313.03  48.895852
